In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.optim import SGD

import plotly.graph_objects as go
import plotly.express as px

from sklearn.metrics import roc_auc_score, average_precision_score

In [2]:
data = pd.read_csv('Processed_data.csv',index_col=0)

In [3]:
data

,id,aki,gender,admission_age,race,heart_rate_min,heart_rate_max,heart_rate_mean,sbp_min,sbp_max,...,inr_min,inr_max,pt_min,pt_max,gcs_min,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable,weight_admit
0,39307659,0,0,78.194169,10,72.0,134.0,97.263158,97.0,127.0,...,1.900000,2.30000,20.00000,24.700000,15.0,6.0,5.0,4.0,0.0,82.0
1,38743306,1,0,65.602396,0,60.0,97.0,84.166667,95.0,143.0,...,1.100000,1.10000,12.10000,12.100000,15.0,6.0,5.0,4.0,0.0,62.1
2,32339865,1,0,64.906629,1,59.0,87.0,71.461538,113.0,150.0,...,1.200000,1.20000,12.80000,12.800000,15.0,1.0,0.0,1.0,1.0,113.1
3,39493447,0,1,45.197789,2,54.0,73.0,63.000000,122.0,142.0,...,1.000000,1.00000,10.50000,10.500000,15.0,6.0,5.0,4.0,0.0,93.8
4,38941409,0,0,44.317598,6,60.0,80.0,65.142857,99.0,144.0,...,1.000000,1.00000,10.60000,10.600000,15.0,6.0,5.0,4.0,0.0,54.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33885,37061810,1,1,71.189233,0,57.0,77.0,65.545455,79.0,122.0,...,1.200000,1.20000,12.60000,12.600000,15.0,6.0,5.0,4.0,0.0,124.0
33886,33431859,0,1,66.174854,0,46.0,104.0,84.652174,99.0,162.0,...,1.100000,1.10000,12.30000,12.300000,13.0,6.0,3.0,4.0,0.0,107.7
33887,38881410,1,0,57.033913,0,94.0,112.0,102.040000,89.0,108.0,...,1.305189,1.52364,14.33761,16.586131,15.0,6.0,0.0,2.0,1.0,44.2
33888,31061555,0,0,55.399450,0,68.0,162.0,91.666667,83.0,132.0,...,0.900000,0.90000,9.60000,10.000000,3.0,1.0,1.0,1.0,0.0,80.0


Use all the features

In [4]:
data1 = data

In [5]:
X_raw = data1.iloc[:,2:]
y_df = data1.iloc[:,1]
X_df = (X_raw - X_raw.mean())/X_raw.std()

In [6]:
X = torch.tensor(X_df.to_numpy(),dtype=torch.float32)
m,n = X.shape
y = torch.tensor(y_df.to_numpy(),dtype=torch.float32).reshape(m,1)

In [7]:
cases = ['train','test']
case_list = np.random.choice(cases,size=X.shape[0],replace=True,p=[0.8,0.2])
X_train = X[case_list=='train']
X_test = X[case_list=='test']
y_train = y[case_list=='train']
y_test = y[case_list=='test']

In [8]:
h = torch.nn.Linear(
    in_features=n,
    out_features=1,
    bias=True
)
sigma = torch.nn.Sigmoid()

f = torch.nn.Sequential(
    h,
    sigma
)

J_BCE = torch.nn.BCELoss()
GD_optimizer = torch.optim.SGD(lr=0.005,params=f.parameters())

nIter = 10000
printInterval = 1000

for i in range(nIter):
    GD_optimizer.zero_grad()
    pred = f(X_train)
    loss = J_BCE(pred,y_train)
    loss.backward()
    GD_optimizer.step()
    if i == 0 or ((i+1)%printInterval) == 0:
        print('Iter {}: average BCE loss is {:.3f}'.format(i+1,loss.item()))

with torch.no_grad():
    pred_test = f(X_test)

auroc = roc_auc_score(y_test,pred_test)
ap = average_precision_score(y_test,pred_test)
print('On test dataset: AUROC {:.3f}, AP {:.3f}'.format(auroc,ap))

Iter 1: average BCE loss is 0.748
Iter 1000: average BCE loss is 0.606
Iter 2000: average BCE loss is 0.598
Iter 3000: average BCE loss is 0.595
Iter 4000: average BCE loss is 0.594
Iter 5000: average BCE loss is 0.593
Iter 6000: average BCE loss is 0.592
Iter 7000: average BCE loss is 0.592
Iter 8000: average BCE loss is 0.592
Iter 9000: average BCE loss is 0.591
Iter 10000: average BCE loss is 0.591
On test dataset: AUROC 0.756, AP 0.751


In [9]:
threshold = 0.5

with torch.no_grad():
    pred_test = f(X_test)

binary_pred = np.where(pred_test.squeeze()>threshold,'positive','negative')
label = np.where(y_test.squeeze()>0.5,'positive','negative')
acc = (binary_pred==label).sum()/binary_pred.shape[0]
print('Accuracy on test dataset is {:.2f}%'.format(acc*100))

Accuracy on test dataset is 67.91%


In [10]:
pd.crosstab(
    index=label,
    columns=binary_pred,
    rownames=['Label'],
    colnames=['Pred']
)

Pred,negative,positive
Label,,
negative,2277,1087
positive,1080,2308


In [11]:
pred_df = pd.DataFrame(
    {
        'pred_probability':pred_test.squeeze(),
        'label':label
    }
)
fig = px.scatter(data_frame=pred_df,y='pred_probability',color='label')
fig

Use features that may be associated with aki throught medical analysis

In [12]:
data2 = data

In [13]:
data2 = data2[['aki',
            'heart_rate_min','heart_rate_max','heart_rate_mean',
            'sbp_min','sbp_max','sbp_mean',
            'dbp_min','dbp_max','dbp_mean',
            'mbp_min','mbp_max','mbp_mean',
            'sodium_min.1','sodium_max.1',
            'potassium_min.1','potassium_max.1',
            'chloride_min.1','chloride_max.1',
            'calcium_min.1','calcium_max.1',
            'bicarbonate_min.1','bicarbonate_max.1',
            'hematocrit_min.1','hematocrit_max.1',
            'hemoglobin_min.1','hemoglobin_max.1',
            'temperature_min','temperature_max','temperature_mean',
            'glucose_min','glucose_max','glucose_mean',
            'bun_min','bun_max']]

In [14]:
data2

,aki,heart_rate_min,heart_rate_max,heart_rate_mean,sbp_min,sbp_max,sbp_mean,dbp_min,dbp_max,dbp_mean,...,hemoglobin_min.1,hemoglobin_max.1,temperature_min,temperature_max,temperature_mean,glucose_min,glucose_max,glucose_mean,bun_min,bun_max
0,0,72.0,134.0,97.263158,97.0,127.0,109.833333,56.0,89.0,70.166667,...,12.1,13.0,36.280000,37.000000,36.558000,127.000000,132.000000,129.500000,23.0,30.0
1,1,60.0,97.0,84.166667,95.0,143.0,112.153846,56.0,99.0,73.307692,...,9.5,9.5,36.670000,37.000000,36.805000,207.000000,305.000000,254.000000,8.0,8.0
2,1,59.0,87.0,71.461538,113.0,150.0,138.160000,60.0,94.0,80.200000,...,16.1,18.6,36.830000,37.280000,37.087143,111.000000,120.000000,117.000000,39.0,39.0
3,0,54.0,73.0,63.000000,122.0,142.0,130.666667,77.0,98.0,89.933333,...,11.8,11.8,36.440000,36.500000,36.470000,150.334439,839.621426,288.537646,18.0,18.0
4,0,60.0,80.0,65.142857,99.0,144.0,125.058824,59.0,111.0,76.000000,...,11.1,11.1,36.500000,37.060000,36.780000,104.000000,139.000000,121.200000,13.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33885,1,57.0,77.0,65.545455,79.0,122.0,96.772727,49.0,92.0,64.636364,...,12.5,12.5,36.440000,36.940000,36.628333,110.000000,177.000000,143.500000,35.0,37.0
33886,0,46.0,104.0,84.652174,99.0,162.0,130.038462,52.0,114.0,73.846154,...,10.7,11.7,36.060000,36.780000,36.348571,108.000000,116.000000,111.000000,12.0,14.0
33887,1,94.0,112.0,102.040000,89.0,108.0,97.500000,54.0,78.0,62.038462,...,9.6,10.2,36.890000,38.780000,37.438750,134.000000,233.000000,162.500000,5.0,10.0
33888,0,68.0,162.0,91.666667,83.0,132.0,113.000000,43.0,86.0,65.000000,...,13.6,13.7,36.440000,37.720000,37.080000,132.000000,132.000000,132.000000,14.0,15.0


In [15]:
X_raw = data2.iloc[:,1:]
y_df = data2.iloc[:,0]
X_df = (X_raw - X_raw.mean())/X_raw.std()

In [16]:
X = torch.tensor(X_df.to_numpy(),dtype=torch.float32)
m,n = X.shape
y = torch.tensor(y_df.to_numpy(),dtype=torch.float32).reshape(m,1)


cases = ['train','test']
case_list = np.random.choice(cases,size=X.shape[0],replace=True,p=[0.8,0.2])
X_train = X[case_list=='train']
X_test = X[case_list=='test']
y_train = y[case_list=='train']
y_test = y[case_list=='test']

In [17]:
h = torch.nn.Linear(
    in_features=n,
    out_features=1,
    bias=True
)
sigma = torch.nn.Sigmoid()

f = torch.nn.Sequential(
    h,
    sigma
)

J_BCE = torch.nn.BCELoss()
GD_optimizer = torch.optim.SGD(lr=0.005,params=f.parameters())

nIter = 20000
printInterval = 1000

for i in range(nIter):
    GD_optimizer.zero_grad()
    pred = f(X_train)
    loss = J_BCE(pred,y_train)
    loss.backward()
    GD_optimizer.step()
    if i == 0 or ((i+1)%printInterval) == 0:
        print('Iter {}: average BCE loss is {:.3f}'.format(i+1,loss.item()))

with torch.no_grad():
    pred_test = f(X_test)

auroc = roc_auc_score(y_test,pred_test)
ap = average_precision_score(y_test,pred_test)
print('On test dataset: AUROC {:.3f}, AP {:.3f}'.format(auroc,ap))

Iter 1: average BCE loss is 0.753
Iter 1000: average BCE loss is 0.637
Iter 2000: average BCE loss is 0.633
Iter 3000: average BCE loss is 0.632
Iter 4000: average BCE loss is 0.631
Iter 5000: average BCE loss is 0.630
Iter 6000: average BCE loss is 0.630
Iter 7000: average BCE loss is 0.630
Iter 8000: average BCE loss is 0.629
Iter 9000: average BCE loss is 0.629
Iter 10000: average BCE loss is 0.629
Iter 11000: average BCE loss is 0.629
Iter 12000: average BCE loss is 0.629
Iter 13000: average BCE loss is 0.628
Iter 14000: average BCE loss is 0.628
Iter 15000: average BCE loss is 0.628
Iter 16000: average BCE loss is 0.628
Iter 17000: average BCE loss is 0.628
Iter 18000: average BCE loss is 0.628
Iter 19000: average BCE loss is 0.628
Iter 20000: average BCE loss is 0.628
On test dataset: AUROC 0.714, AP 0.699


In [18]:
threshold = 0.5

with torch.no_grad():
    pred_test = f(X_test)

binary_pred = np.where(pred_test.squeeze()>threshold,'positive','negative')
label = np.where(y_test.squeeze()>0.5,'positive','negative')
acc = (binary_pred==label).sum()/binary_pred.shape[0]
print('Accuracy on test dataset is {:.2f}%'.format(acc*100))

Accuracy on test dataset is 65.97%


In [19]:
pd.crosstab(
    index=label,
    columns=binary_pred,
    rownames=['Label'],
    colnames=['Pred']
)

Pred,negative,positive
Label,,
negative,2266,1050
positive,1230,2153


In [20]:
weight = h.weight.detach().squeeze().clone()

In [21]:
h_L2 = torch.nn.Linear(
    in_features=n,
    out_features=1,
    bias=True
)
sigma = torch.nn.Sigmoid()

# Logistic model is linear+sigmoid
f_L2 = torch.nn.Sequential(
    h_L2,
    sigma
)

J_BCE = torch.nn.BCELoss()

# PyTorch optimizer support L2 regularization by
# setting the weight_decay parameter, which corresponds to
# the regularization strength.
GD_optimizer = torch.optim.Adam(lr=0.01,params=f_L2.parameters(),weight_decay=0.05)

nIter = 5000
printInterval = 50

for i in range(nIter):
    GD_optimizer.zero_grad()
    pred = f_L2(X_train)
    loss = J_BCE(pred,y_train)
    loss.backward()
    GD_optimizer.step()
    if i == 0 or ((i+1)%printInterval) == 0:
        print('Iter {}: average BCE loss is {:.3f}'.format(i+1,loss.item()))

with torch.no_grad():
    pred_test = f_L2(X_test)

auroc = roc_auc_score(y_test,pred_test)
ap = average_precision_score(y_test,pred_test)
print('On test dataset: AUROC {:.3f}, AP {:.3f}'.format(auroc,ap))

Iter 1: average BCE loss is 0.768
Iter 50: average BCE loss is 0.634
Iter 100: average BCE loss is 0.634
Iter 150: average BCE loss is 0.634
Iter 200: average BCE loss is 0.634
Iter 250: average BCE loss is 0.634
Iter 300: average BCE loss is 0.634
Iter 350: average BCE loss is 0.634
Iter 400: average BCE loss is 0.634
Iter 450: average BCE loss is 0.634
Iter 500: average BCE loss is 0.634
Iter 550: average BCE loss is 0.634
Iter 600: average BCE loss is 0.634
Iter 650: average BCE loss is 0.634
Iter 700: average BCE loss is 0.634
Iter 750: average BCE loss is 0.634
Iter 800: average BCE loss is 0.634
Iter 850: average BCE loss is 0.634
Iter 900: average BCE loss is 0.634
Iter 950: average BCE loss is 0.634
Iter 1000: average BCE loss is 0.634
Iter 1050: average BCE loss is 0.634
Iter 1100: average BCE loss is 0.634
Iter 1150: average BCE loss is 0.634
Iter 1200: average BCE loss is 0.634
Iter 1250: average BCE loss is 0.634
Iter 1300: average BCE loss is 0.634
Iter 1350: average BCE l

In [22]:
threshold = 0.5

with torch.no_grad():
    pred_test = f_L2(X_test)

binary_pred = np.where(pred_test.squeeze()>threshold,'positive','negative')
label = np.where(y_test.squeeze()>0.5,'positive','negative')
acc = (binary_pred==label).sum()/binary_pred.shape[0]
print('Accuracy on test dataset is {:.2f}%'.format(acc*100))

Accuracy on test dataset is 65.83%


In [23]:
pd.crosstab(
    index=label,
    columns=binary_pred,
    rownames=['Label'],
    colnames=['Pred']
)

Pred,negative,positive
Label,,
negative,2255,1061
positive,1228,2155


In [24]:
weight_L2 = h_L2.weight.detach().squeeze().clone()

In [25]:
h_L1 = torch.nn.Linear(
    in_features=n,
    out_features=1,
    bias=True
)
sigma = torch.nn.Sigmoid()

# Logistic model is linear+sigmoid
f_L1 = torch.nn.Sequential(
    h_L1,
    sigma
)

J_BCE = torch.nn.BCELoss()

GD_optimizer = torch.optim.Adam(lr=0.01,params=f_L1.parameters())

# Define L_1 regularization
def L1_reg(model,lbd):
    result = torch.tensor(0)
    for param in model.parameters(): # iterate over all parameters of our model
        result = result + param.abs().sum()

    return lbd*result


nIter = 5000
printInterval = 50
lbd = 0.03 # L1 reg strength

for i in range(nIter):
    GD_optimizer.zero_grad()
    pred = f_L1(X_train)
    loss = J_BCE(pred,y_train)
    (loss+L1_reg(f_L1,lbd)).backward()
    GD_optimizer.step()
    if i == 0 or ((i+1)%printInterval) == 0:
        print('Iter {}: average BCE loss is {:.3f}'.format(i+1,loss.item()))

with torch.no_grad():
    pred_test = f_L1(X_test)

auroc = roc_auc_score(y_test,pred_test)
ap = average_precision_score(y_test,pred_test)
print('On test dataset: AUROC {:.3f}, AP {:.3f}'.format(auroc,ap))

Iter 1: average BCE loss is 0.712
Iter 50: average BCE loss is 0.654
Iter 100: average BCE loss is 0.652
Iter 150: average BCE loss is 0.652
Iter 200: average BCE loss is 0.652
Iter 250: average BCE loss is 0.651
Iter 300: average BCE loss is 0.652
Iter 350: average BCE loss is 0.652
Iter 400: average BCE loss is 0.652
Iter 450: average BCE loss is 0.652
Iter 500: average BCE loss is 0.652
Iter 550: average BCE loss is 0.652
Iter 600: average BCE loss is 0.652
Iter 650: average BCE loss is 0.652
Iter 700: average BCE loss is 0.653
Iter 750: average BCE loss is 0.652
Iter 800: average BCE loss is 0.652
Iter 850: average BCE loss is 0.651
Iter 900: average BCE loss is 0.652
Iter 950: average BCE loss is 0.652
Iter 1000: average BCE loss is 0.652
Iter 1050: average BCE loss is 0.652
Iter 1100: average BCE loss is 0.652
Iter 1150: average BCE loss is 0.652
Iter 1200: average BCE loss is 0.652
Iter 1250: average BCE loss is 0.652
Iter 1300: average BCE loss is 0.652
Iter 1350: average BCE l

In [26]:
threshold = 0.5

with torch.no_grad():
    pred_test = f_L1(X_test)

binary_pred = np.where(pred_test.squeeze()>threshold,'positive','negative')
label = np.where(y_test.squeeze()>0.5,'positive','negative')
acc = (binary_pred==label).sum()/binary_pred.shape[0]
print('Accuracy on test dataset is {:.2f}%'.format(acc*100))

Accuracy on test dataset is 64.56%


In [27]:
pd.crosstab(
    index=label,
    columns=binary_pred,
    rownames=['Label'],
    colnames=['Pred']
)

Pred,negative,positive
Label,,
negative,2274,1042
positive,1332,2051


In [28]:
weight_L1 = h_L1.weight.detach().squeeze().clone()

In [29]:
weight_df = pd.DataFrame(
    {
        'vanilla':weight,
        'L2':weight_L2,
        'L1':weight_L1
    }
).melt(id_vars=[],value_vars=['vanilla','L2','L1'])
weight_df

,variable,value
0,vanilla,0.064485
1,vanilla,0.099192
2,vanilla,-0.051782
3,vanilla,-0.256935
4,vanilla,0.286850
...,...,...
97,L1,-0.002967
98,L1,0.000351
99,L1,-0.004029
100,L1,0.167005


In [30]:
fig = px.box(
    weight_df,
    y='value',
    facet_col='variable',
    color='variable',
    points='all',
    title='Logistic Regression Weights Distributions'
)
fig.update_yaxes(
    matches=None,
    showticklabels=True
)
fig.update_traces(jitter=0.5)

Use Forward and Backward feature selection to select features

In [31]:
from sklearn.linear_model import LogisticRegression as logit
from sklearn.feature_selection import SequentialFeatureSelector as SFS
from sklearn.metrics import roc_curve, precision_recall_curve

In [32]:
X_df_train = X_df.iloc[case_list=='train',:]
X_df_test = X_df.iloc[case_list=='test',:]
y_df_train = y_df.iloc[case_list=='train']
y_df_test = y_df.iloc[case_list=='test']

In [33]:
model = logit(penalty='l1',C=1/10,solver='liblinear')

In [34]:
# Forward feature selection.
forward_selection = SFS(
    model, n_features_to_select=3, direction="forward"
).fit(X_df_train, y_df_train)

In [35]:
# Backward feature selection.
backward_selection = SFS(
    model, n_features_to_select=3, direction="backward"
).fit(X_df_train, y_df_train)

In [36]:
forward_selection.get_feature_names_out()

array(['sbp_min', 'potassium_max.1', 'bun_min'], dtype=object)

In [37]:
backward_selection.get_feature_names_out()

array(['sbp_min', 'sbp_max', 'bun_min'], dtype=object)

In [38]:
model.fit(X_df_train,y_df_train)
y_pred_full = model.predict_proba(X_df_test)

model.fit(forward_selection.transform(X_df_train),y_df_train)
y_pred_FS = model.predict_proba(forward_selection.transform(X_df_test))

model.fit(backward_selection.transform(X_df_train),y_df_train)
y_pred_BS = model.predict_proba(backward_selection.transform(X_df_test))

In [39]:
# roc_curve
fpr_full, tpr_full, _ = roc_curve(y_df_test,y_pred_full[:,1])
fpr_FS, tpr_FS, _ = roc_curve(y_df_test,y_pred_FS[:,1])
fpr_BS, tpr_BS, _ = roc_curve(y_df_test,y_pred_BS[:,1])

roc_df = pd.DataFrame(
    {
        'False Positive Rate':np.hstack([fpr_full,fpr_FS,fpr_BS]),
        'True Positive Rate':np.hstack([tpr_full,tpr_FS,tpr_BS]),
        'method':['full_model']*len(fpr_full)+['FS']*len(fpr_FS)+['BS']*len(fpr_BS)
    }
)

In [40]:
# Visualize ROC curve
fig = px.line(roc_df,y='True Positive Rate',x='False Positive Rate',facet_col='method',color='method')
fig

In [41]:
# precision recall curves
p_full, r_full, _ = precision_recall_curve(y_df_test,y_pred_full[:,1])
p_FS, r_FS, _ = precision_recall_curve(y_df_test,y_pred_FS[:,1])
p_BS, r_BS, _ = precision_recall_curve(y_df_test,y_pred_BS[:,1])

pr_df = pd.DataFrame(
    {
        'Precision':np.hstack([p_full,p_FS,p_BS]),
        'Recall':np.hstack([r_full,r_FS,r_BS]),
        'method':['Full Model']*len(p_full)+['Forward Selection']*len(p_FS)+['Backward Selection']*len(p_BS)
    }
)

In [42]:
fig = px.line(pr_df,x='Recall',y='Precision',facet_col='method',color='method')
fig

In [43]:
auroc = roc_auc_score(y_df_test, y_pred_FS[:, 1])
ap = average_precision_score(y_df_test, y_pred_FS[:, 1])

print('Forward Selection')
print("AUROC:", auroc)
print("AP:", ap)

Forward Selection
AUROC: 0.6717134241419258
AP: 0.6572300966494881


In [44]:
auroc = roc_auc_score(y_df_test, y_pred_BS[:, 1])
ap = average_precision_score(y_df_test, y_pred_BS[:, 1])

print('Backward Selection')
print("AUROC:", auroc)
print("AP:", ap)

Backward Selection
AUROC: 0.6810947521257746
AP: 0.6596578558219695
